In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    HierarchyPaths, FormulaTableTypeSimple)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies())
formulas_by_year = Formulas(data_source.formulas_by_year())
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative())

In [11]:
index_formulas = formulas_by_year.filter_by_function(lambda x: x['Category'] == 'Indices')
my_table = FormulaTable(index_formulas)

my_table.set_type(FormulaTableTypeSimple)

my_table.contains_content()

True

In [15]:

def _calclus_summary_comment(row):
    """Returns a comment for calculus formula summary based on
    derivative and integral comments
    """
    if row['Comment Differentiation'] == row['Comment Integration']:
        return_value = row['Comment Differentiation']
    elif row['Comment Differentiation'] == '':
        return_value = row['Comment Integration']
    elif row['Comment Integration'] == '':
        return_value = row['Comment Differentiation']
    else:
        return_value = (row['Comment Differentiation'] + '\n' +
                        row['Comment Integration'])
    return return_value


def get_calculus_summary_df(formulas_df):
    """Returns a summary of derivative and integral formulas

    Args:
        formulas_df (pandas dataframe): The source of fomulas for producing the
        summary

    Returns:
        pandas dataframe: Summary of calculus and equicalent integraml
        formulas.Returns None if no derivative of integration formulas
        exists in formulas_input_df
    """
    calculus_df = (formulas_df
                   [['Category', 'Group', 'Formula', 'Comment']]
                   [formulas_df["Category"].isin(["Differentiation", "Integration"])])
    if len(calculus_df.index) == 0:
        return None
    calculus_df = calculus_df.pivot(
        columns='Category', index='Group').fillna('')

    # Flatten the multi-index headings after pivot
    calculus_df.columns = (
        calculus_df.columns.get_level_values(0) + ' ' +
        calculus_df.columns.get_level_values(1))
    calculus_df = calculus_df.reset_index()

    calculus_df['Comment'] = (
        calculus_df.apply(_calclus_summary_comment, axis=1))

    calculus_df = calculus_df.sort_values(by='Group')
    calculus_df = calculus_df.drop(
        labels=['Group', 'Comment Differentiation', 'Comment Integration'],
        axis=1)
    calculus_df = calculus_df.rename(columns={
        "Formula Differentiation": "Derivative",
        "Formula Integration": "Equivalent integral"})

    # Reorder columns
    calculus_df = calculus_df[['Derivative', 'Equivalent integral',
                               'Comment']]
    return calculus_df




In [ ]:
df = formulas_by_year
display(get_calculus_summary_df(formulas_by_year.to_dataframe()))

In [30]:
from abc import ABC, abstractmethod



class FormulaTableTypeOne(FormulaTableType):

    def formula_columns(self):
        return 1

    def to_dataframe(self):
        return 2


a = FormulaTableTypeOne(formulas_by_year)

a.formula_columns()


1

In [18]:
from abc import ABC, abstractmethod
 
class AbstractClassExample(ABC):
 
    def __init__(self, value):
        self.value = value
        super().__init__()
    
    @abstractmethod
    def do_something(self):
        pass

class DoAdd42(AbstractClassExample):

    def do_something(self):
        return self.value + 42
    
class DoMul42(AbstractClassExample):
   
    def do_something(self):
        return self.value * 42
    
x = DoAdd42(10)
y = DoMul42(10)

print(x.do_something())
print(y.do_something())

52
420
